In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import mysql.connector
from mysql.connector import Error

In [2]:
def extraer_datos(ruta_de_archivo): 
    archivo=pd.read_csv(ruta_de_archivo)
    data_frame=pd.DataFrame(archivo)
    return(data_frame)

In [3]:
def transformacion(data_frame):
    fecha=[]
    precio=[]
    for i in range(len(data_frame['Fecha de venta'])):
        date_str=data_frame['Fecha de venta'][i]+' '+data_frame['Hora de Venta'][i]
        date_format= '%m/%d/%Y %I:%M %p'
        date_obj = datetime.strptime(date_str, date_format)
        precio_obj = int(data_frame['Precio de venta'][i]/data_frame['Cantidad vendida'][i])
        fecha.append(date_obj)
        precio.append(precio_obj)

    ventas = pd.DataFrame({'id':data_frame['ID de venta'],'fecha':fecha,'id_producto':data_frame['ID de producto'],
                                         'id_cliente':data_frame['ID de cliente'],'cantidad':data_frame['Cantidad vendida'],
                                         'descuento':data_frame['Descuento']
                                        ,'id_tienda':data_frame['ID de la tienda']}
                               )
    productos = pd.DataFrame({
                              'id':data_frame['ID de producto'],
                              'nombre':data_frame['Nombre del producto'],
                                'precio':precio
    })
    clientes= pd.DataFrame({
            'id':data_frame['ID de cliente'],
            'nombre':data_frame['Nombre del cliente'],
            'genero':data_frame['Género del cliente']
            
    })
    tiendas = pd.DataFrame({
        'id':data_frame['ID de la tienda'],
        'nombre':data_frame['Nombre de la tienda'],
        'ubicacion':data_frame['Ubicación de la tienda'].fillna('no-info')
    })

    return(ventas,productos,clientes,tiendas)

In [4]:
def guardar_csv(data_frame,archivo_destino):
    data_frame.to_csv(archivo_destino,index=False)

In [5]:
def crearTablas(nombrehost,puerto,usuario,contraseña,database):
    try:
        conexion = mysql.connector.connect(
            host = nombrehost,
            port = puerto,
            user = usuario,
            password = contraseña,
            db= database
        )
        if conexion.is_connected():
            print('Base de datos conectada')

            cursor = conexion.cursor()
            cursor.execute('''
                        CREATE TABLE IF NOT EXISTS clientes (
                        id VARCHAR(100) NOT NULL,
                        nombre VARCHAR(80) NOT NULL,
                        genero VARCHAR(20),
                        PRIMARY KEY (id)
                        )
                        ''')
            cursor.execute('''
                        CREATE TABLE IF NOT EXISTS productos (
                        id VARCHAR(100) NOT NULL,
                        nombre VARCHAR(80) NOT NULL,
                        precio INT(20) NOT NULL,
                        PRIMARY KEY (id)
                        )
                        ''')
            cursor.execute('''CREATE TABLE IF NOT EXISTS tiendas (
                        id VARCHAR(100) NOT NULL,
                        nombre VARCHAR(80) NOT NULL,
                        ubicacion VARCHAR(80),
                        PRIMARY KEY (id)
                        )''')
            cursor.execute("""CREATE TABLE IF NOT EXISTS ventas (
                        id VARCHAR(100) NOT NULL,
                        fecha DATETIME NOT NULL,
                        id_producto VARCHAR(100) NOT NULL,
                        id_cliente VARCHAR(100) NOT NULL,
                        cantidad INT(10) NOT NULL,
                        descuento INT(10) NOT NULL,
                        id_tienda VARCHAR(100) NOT NULL,
                        PRIMARY KEY (id),
                        FOREIGN KEY (id_producto)
                        REFERENCES productos(id) ON UPDATE CASCADE,
                        FOREIGN KEY (id_cliente)
                        REFERENCES clientes(id) ON UPDATE CASCADE,
                        FOREIGN KEY (id_tienda)
                        REFERENCES tiendas(id) ON UPDATE CASCADE                       
                        )
                        """)
            
            print('Tablas creadas')

    except Error as ex:
        print('Error al conectar base de datos: {}'.format(ex))
    finally:
        if conexion.is_connected():
            conexion.close()
            print('Conexión finalizada')

In [6]:
def guardarDatos(sql,tabla):
    try:
        conexion = mysql.connector.connect(
            host = 'localhost',
            port = 3306,
            user = 'root',
            password = 'valkiria123',
            db= 'minoristaelectronicos'
        )
        if conexion.is_connected():
            cursor = conexion.cursor()

            cursor.executemany(sql,tabla)
            conexion.commit()           
            


    except Error as ex:
        print('Error al conectar base de datos: {}'.format(ex))
    finally:
        if conexion.is_connected():
            conexion.close()

In [7]:
ruta='sales_data.csv'
datos_extraidos=extraer_datos(ruta)
print('Datos extraídos')
tabla_ventas , tabla_productos , tabla_clientes, tabla_tiendas =transformacion(datos_extraidos)
print('Datos transformados')
crearTablas('localhost',3306,'root','valkiria123','minoristaelectronicos')
sql_productos='INSERT INTO productos (id,nombre,precio) VALUES (%s,%s,%s)'
sql_clientes='INSERT INTO clientes (id,nombre, genero) VALUES (%s,%s,%s)'
sql_tiendas='INSERT INTO tiendas (id,nombre, ubicacion) VALUES (%s,%s,%s)'
sql_ventas='INSERT INTO ventas (id,fecha,id_producto,id_cliente,cantidad,descuento,id_tienda) VALUES (%s,%s,%s,%s,%s,%s,%s)'
guardarDatos(sql_productos,tabla_productos.to_numpy().tolist())
guardarDatos(sql_clientes,tabla_clientes.to_numpy().tolist())
guardarDatos(sql_tiendas,tabla_tiendas.to_numpy().tolist())
guardarDatos(sql_ventas,tabla_ventas.to_numpy().tolist())
print('datos insertados')

Datos extraídos
Datos transformados
Base de datos conectada
Tablas creadas
Conexión finalizada
datos insertados


In [8]:
ruta='sales_data.csv'
datos_extraidos=extraer_datos(ruta)
print('Datos extraídos')
tabla_ventas , tabla_productos , tabla_clientes, tabla_tiendas =transformacion(datos_extraidos)
print('Datos transformados')
guardar_csv(tabla_ventas,'ventas.csv')
guardar_csv(tabla_productos,'productos.csv')
guardar_csv(tabla_clientes,'clientes.csv')
guardar_csv(tabla_tiendas,'tiendas.csv')
print('datos guardados')

Datos extraídos
Datos transformados
datos guardados
